<a href="https://colab.research.google.com/github/MichaelPaulukonis/notebooks/blob/main/min_dalle_(version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# min(DALL·E)

In [ ]:
#@title Save to Google Drive
#@markdown Will save your results to Google Drive - image and prompt+settings

save_to_google_drive = True  #@param {type: "boolean"}

if save_to_google_drive:
  from google.colab import drive
  import time
  
  #@title Google Colab Google Drive Downloader Thing { vertical-output: true }
  Target_Folder = "/content/drive/MyDrive/colab_out/" #@param {type:"string"}
  drive.mount('/content/drive')
  
  !mkdir -p $Target_Folder

  save_prompt = True  #@param {type: "boolean"}
  prompt_file = "/content/drive/MyDrive/colab_out/prompts.txt" #@param {type:"string"}


### Install

In [ ]:
! nvidia-smi
! pip install min-dalle==0.4.7
! pip install Pillow

### Load Model
`float32` is faster than `float16` but uses more GPU memory.

In [ ]:
dtype = "float16" #@param ["float32", "float16"]
from IPython.display import display, update_display
import torch
from min_dalle import MinDalle

model = MinDalle(
    dtype=getattr(torch, dtype),
    device='cuda',
    is_mega=True, 
    is_reusable=True
)

### Generate Images

- `image_prefix` name of the images (if you're saving to Google Drive)

- `progressive_outputs` Whether to show intermediate output.  Adds a small delay and increases memory usage.

- `seamless` Tiles the images in token space instead of pixel space

- `grid_size` Size of the image grid.  Reduce this if you run out of GPU memory.

- `batch_size` Number of images to generate. If you're not saving to GDrive, this is somewhat pointless.

- `temperature` High temperature increases the probability of sampling low scoring image tokens.

- `supercondition_factor` Higher values result in better agreement with the text but a narrower variety of generated images

- `top_k` Each image token is sampled from the top $k$ most probable tokens

- `use_random_seed` Using the built-in random seed results in different images each time. Set manually to get results starting from the same seed. If running in batch mode `use_random_seed` is silently enabled.

- `manual_seed` An integer from 1 to 10,000. If running in batch mode `manual_seed` is ignored.

- `is_verbose` min-dalle will print out extra internal data (mostly about text tokenization)

In [ ]:
%%time

text = "vibrant popart screenprint of a human anatomical chart diagram with labels, by Andy Warhol; strong bold solid dayglo colors, popart, deep vibrant intense contrasty color" #@param {type:"string"}
if save_to_google_drive:
  image_prefix = "tests" #@param {type:"string"}
progressive_outputs = False #@param {type:"boolean"}
seamless = False #@param {type:"boolean"}
grid_size =  1 #@param {type:"integer"}
batch_size = "2" #@param [1,2,3,4,5,6,7,8,9,10,15,20]
temperature = 16 #@param {type:"slider", min:0.01, max:16, step:0.01}
supercondition_factor =  "64" #@param [4, 8, 16, 32, 64]
top_k =  "4" #@param [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]
use_random_seed = False #@param {type:"boolean"}
if not use_random_seed:
  manual_seed = 14 #@param {type:"integer"}
is_verbose = False #@param {type:"boolean"}

batch_size = int(batch_size)
if batch_size > 1: use_random_seed = True

if save_to_google_drive:
  !mkdir -p '{Target_Folder}/{image_prefix}'

if save_prompt:
  with open(prompt_file, 'a') as f:
    f.write(f'{text} {{ temp: {temperature}, sf: {supercondition_factor} top_k: {top_k}, start: {time.strftime("%Y%m%d-%H%M%S")} }}\n')

for x in range(0, batch_size):
  image = model.generate_image(
      text=text,
      seed= -1 if use_random_seed else manual_seed,
      grid_size=grid_size,
      is_seamless=seamless,
      temperature=temperature,
      top_k=int(top_k),
      supercondition_factor=float(supercondition_factor),
      is_verbose=is_verbose
  )

  if batch_size > 1:
    print(f"\r{x+1} of {batch_size}")

  display_image = display if (x == 0) else update_display
  display_image(image, display_id=1)

  if save_to_google_drive:
    timestr = time.strftime("%Y%m%d-%H%M%S")
    image.save(f'{Target_Folder}/{image_prefix}/{image_prefix}_{timestr}_{temperature}_{supercondition_factor}_{top_k}.png')